<a href="https://colab.research.google.com/github/infinitylogesh/Interpretable-NLP-Talk/blob/master/Dathack_2019_LSTM_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports


In [0]:
import torch,sys,cPickle
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Attention Intuition :

Attention :

In an attention layer, the weight of each token is computed by an alignment model which scores how well the inputs contribute to the classification or the output (in case of seq-seq). An alignment model is a feedforward neural network, for instance. In general, it can be any other model as well.
As a result, the alphas — the weights of hidden states when computing a context vector — show how important a given Token.


![alt text](https://i.imgur.com/N7D7rT9.png)

After Attention :

![alt text](https://i.imgur.com/IGfgRdZ.png)

# Token-level importance visualization with Attention weights


We use a one layer feed forward style attention ([Bahdanau et al., 2015 ](https://doi.org/10.1146/annurev.neuro.26.041002.131047)) on top of LSTMs to get the token level importance scores. We use these scores in our visualizations. 

#### Attention :

\begin{align}\begin{split} 
\mathbf{c}_i &= \sum\limits_j a_{ij}\mathbf{s}_j\\ 
\mathbf{a}_i &= \text{softmax}(f_{att}(\mathbf{s}_j)) 
\end{split}\end{align}



### Model Architecture.

We train a model as shown below on Yelp sentiment analysis dataset to extract the token level importance score.

-


> ![BiLSTM-Attnetion](https://i.imgur.com/K1NGVcu.png)











In [0]:
sys.path.insert(0, './drive/My Drive/talks/Interpretable_NLP/Attention/')

In [0]:
import keras
from attention import Attention
from custom_metrics import f1
from pre_process import clean_text,tokenize
from keras import models
from keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import joblib
import numpy as np
import tensorflow as tf
from flask import jsonify
import logging
import pandas as pd
from keras import backend as K
from keras.models import Sequential,Model,load_model
from keras.layers import Input,Reshape
from keras.layers import Dense,Dropout,Flatten,Activation,GlobalAveragePooling2D,LSTM,Bidirectional
from keras.layers import BatchNormalization, Convolution2D, Input,Dropout
from keras.layers import Conv1D,Dense, MaxPooling1D,CuDNNLSTM,CuDNNGRU,SpatialDropout1D,Bidirectional,GlobalAveragePooling1D,LSTM
from keras.layers import Embedding
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.optimizers import Adam,SGD,RMSprop
from keras.layers import Conv1D, MaxPooling1D,CuDNNLSTM,CuDNNGRU,LSTM,GRU
from keras.layers.embeddings import Embedding
import time,os

In [0]:
def model_with_attention_weights(embedding_matrix):
        # https://www.kaggle.com/morenoh149/keras-imagedatagenerator-validation-split
    inp = Input(shape=(200,))
    emb = Embedding(100001, 300, weights=[embedding_matrix], trainable=False)(inp)
    sDropout = SpatialDropout1D(0.1)(emb)
    lstm = Bidirectional(LSTM(64,return_sequences=True,recurrent_activation='sigmoid'))(sDropout)
    lstm = Bidirectional(LSTM(64,return_sequences=True,recurrent_activation='sigmoid'))(lstm)
    attention,attn_w = Attention(200,return_attention_weights=True)(lstm)
    dense_1 = Dense(128, activation="relu")(attention)
    outp = Dense(3, activation="softmax")(dense_1)
    model = Model(inputs=inp, outputs=[outp,attn_w])
    return model

In [7]:
!tar -xvzf drive/My\ Drive/talks/Interpretable_NLP/Attention/artifacts.tar.gz -C .

artifacts/
artifacts/graph/
artifacts/graph/model.10-0.905-0.255-0.899.hdf5
artifacts/pickles/
artifacts/pickles/vocab_top_100000_90_percent_model.txt
artifacts/pickles/embedding_matrix_100001.npy
artifacts/README.txt


In [0]:
import html
from IPython.core.display import display, HTML

def html_escape(text):
    return html.escape(text)

#### Loading all the artifacts - Pretrained model, Vocab and embeddings


In [0]:
model_weights = "artifacts/graph/model.10-0.905-0.255-0.899.hdf5"
vocab_file =  "artifacts/pickles/vocab_top_100000_90_percent_model.txt"
embedding_matrix =  np.load("artifacts/pickles/embedding_matrix_100001.npy")
min_percentile = 70
vocab = pd.read_fwf(vocab_file, names=["words"],widths=[200]) #widths is important otherwise strings are truncated.


In [10]:
model = model_with_attention_weights(embedding_matrix)
model.load_weights(model_weights)
graph = tf.get_default_graph()


W1113 17:01:04.132328 140653462620032 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1113 17:01:04.142267 140653462620032 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1113 17:01:04.152224 140653462620032 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1113 17:01:04.165747 140653462620032 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1113 17:01:04.166631 140653462620032 module_wrapper

### Visualizing the attention weights 

We select only the top 'n' percentile of weights that are returned from the attention layer. Value 'n' can be controlled with the slider. 

The words highlighted in the example are the ones that are given importance for prediction

In [13]:
#@title Attention visualization

labels = ["negative","neutral","positive"]

def _prepare_text(texts):
        processed_texts = []
        clean_texts = []
        temp_vocab = {}
        for text in texts:
                text = text.strip()
                cleaned_text,text_ids,_ = clean_text(text,vocab,temp_vocab)
                processed_texts.append(text_ids)
                clean_texts.append(cleaned_text)
        return processed_texts,clean_texts

   
text = "I love this place. The portions are generous and the plates are fairly priced ($7 to $10 for a plate that includes rice, meats, hummus, and taboule). The workers are always courteous." #@param {type:"string"}
texts,clean_texts = _prepare_text([text])
preds,attn_scores = model.predict(np.array(texts))

max_alpha = 0.8 
min_percentile = 60 #@param {type:"slider", min:0, max:100, step:10}
highlighted_text = []

tokens = text.split()
weights = np.squeeze(attn_scores[0])[:len(tokens)]
weights = (weights > np.percentile(weights,min_percentile)).astype(int)
# max_alpha = np.percentile(weights,min_percentile)

  
for word,weight in zip(clean_texts[0].split(),weights):
    if weight is not None:
        highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(str(word)) + '</span>')
    else:
        highlighted_text.append(str(word))
highlighted_text = ' '.join(highlighted_text)


print("Prediction : {}".format(labels[np.argmax(preds[0])])+"\n\n")

display(HTML(highlighted_text))

Prediction : positive




# Sequence level importance visualization with Segmentation Attention

We use a model ([Bailin et el. 2019 ](http://www.statnlp.org/wp-content/uploads/papers/2018/Learning-Latent/absa.pdf)) with the architecture as shown below and train it on a target sentiment analysis task. The segmentation attention provided by CRF helps in extracting opinions that was used by the model in predicting sentiment of the given target word.

1. This method is shown to be capable of extracting opinions from sentences that carry multiple targets.

2. Authors have used a layer that is analogous to conditional random
field (CRF) in the
attention modeling process to capture the structural dependencies.The resulting new attention mechanism will be able
to perform soft selections of opinion expressions in the form
of word spans. This can be viewed as an extension of the
standard attention mechanism.Authors call this attention mechanism as segmentation attention

-
-

### **Task** : Target Based sentiment Analysis

<img src="https://i.imgur.com/pHCkAbd.png">

### **Model Architecture**

![Model](https://i.imgur.com/C05Y7Au.png)

In [3]:
#!rm -R SA-Sent
!git clone https://github.com/infinitylogesh/SA-Sent.git

Cloning into 'SA-Sent'...
remote: Enumerating objects: 25, done.
remote: Total 25 (delta 0), reused 0 (delta 0), pack-reused 25
Unpacking objects: 100% (25/25), done.


In [12]:
!ls drive/My\ Drive/talks/Interpretable_NLP/Attention_CRF/

data.pkl  dic.pkl  pre-trained-glove.pkl  target_model_best.pt


In [0]:
attention_crf_path = "drive/My Drive/talks/Interpretable_NLP/Attention_CRF"

In [5]:

sys.path.insert(0, './SA-Sent')
model = torch.load(attention_crf_path+"/target_model_best.pt")

/usr/local/lib/python2.7/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'model.AspectSent' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python2.7/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python2.7/dist-packages/torch/serialization.py:493: SourceChangeWarning: source code of class 'torch.nn.modules.dropout.Dropout' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module

In [15]:
model

AspectSent(
  (cat_layer): SimpleCat(
    (word_embed): Embedding(5135, 300)
    (mask_embed): Embedding(2, 50)
    (dropout): Dropout(p=0.4, inplace=False)
  )
  (lstm): MLSTM(
    (rnn): LSTM(350, 100, batch_first=True, dropout=0.1, bidirectional=True)
  )
  (feat2tri): Linear(in_features=200, out_features=2, bias=True)
  (inter_crf): LinearCRF()
  (feat2label): Linear(in_features=200, out_features=3, bias=True)
  (cri): CrossEntropyLoss()
)

In [0]:
import spacy
nlp = spacy.load("en")
tokenizer = nlp.Defaults.create_tokenizer(nlp)


dic = cPickle.load(open(attention_crf_path+"/dic.pkl","r"))
id2vec = cPickle.load(open(attention_crf_path+"/pre-trained-glove.pkl","r"))
data = cPickle.load(open(attention_crf_path+"/data.pkl","r"))
 

id2word = list(dic)
word2id = {v:k for k,v in enumerate(id2word)}

In [0]:
def tokenize(text):
  text = " ".join(text.split("-"))
  text = " ".join(text.split("/"))
  text = " ".join(text.split("!"))
  return list(nlp(unicode(text)))

def vectorizer(text):
  sent_ids = [word2id[str(token)] if str(token) in word2id else word2id["UNK"] for token in tokenize(text)]
  return sent_ids

def prepare_mask(review,target):
  mask = [0]*len(list(tokenize(review)))
  for target_token in  target.split(" "):
    for i,token in enumerate(tokenize(review)):
      if str(token) == target_token :
        mask[i] = 1
  return mask


  
  
  
  

In [0]:
import html
from IPython.core.display import display, HTML

def html_escape(text):
    return html.escape(text)

In [14]:
#@title Attention with CRF

import warnings
warnings.filterwarnings("ignore", category=UserWarning) 

review = "the food is usually good certainly isn't a relaxing place to go" #@param ["the food was bad but the waiter was very nice","the food is usually good but certainly isn't a relaxing place to go","the food is usually good certainly isn't a relaxing place to go","Great beer selection too with 50 beers in the list"] {type:"string"}
target = "place" #@param {type:"string"}
mask = prepare_mask(review,target)
sent_ids = vectorizer(review)

labels = ["positive", "neutral", "negative"]
assert len(list(tokenize(review))) == len(mask)

model.eval()
prediction , activated_sequences =  model.predict(sent_ids,mask)

max_alpha = 0.8 
highlighted_text = []
for word,weight in zip(tokenize(review),activated_sequences):
   
    if weight is not None:
        highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(str(word)) + '</span>')
    else:
        highlighted_text.append(str(word))
highlighted_text = ' '.join(highlighted_text)


print("Predcition: {} \n".format(labels[prediction]))

display(HTML(highlighted_text))

Predcition: positive 



In [0]:
# the food was bad but the waiter was very nice
# the food is usually good but certainly isn't a relaxing place to go 
# Intimate but charming interior with extremely friendly and attentive service . - Service
# Great beer selection too with 50 beers in the list - beer

9